# Effective TensorFlow
Table of Contents

1. TensorFlow Basics
2. Understanding static and dynamic shapes
3. Scopes and when to use them
4. Broadcasting the good and the ugly
5. Feeding data to TensorFlow
6. Take advantage of the overloaded operators
7. Understanding order of execution and control dependencies
8. Control flow operations: conditionals and loops
9. Prototyping kernels and advanced visualization with Python ops
10. Multi-GPU processing with data parallelism
11. Debugging TensorFlow models
12. Numerical stability in TensorFlow
13. Building a neural network training framework with learn API
14. TensorFlow Cookbook
    - Get shape
    - Batch gather
    - Beam search
    - Merge
    - Entropy
    - KL-Divergence
    - Make parallel
    - Leaky Relu
    - Batch normalization
    - Squeeze and excitation


## TensorFlow Basics

The most striking difference between TensorFlow and other numerical computation libraries such as NumPy is that operations in TensorFlow are symbolic. This is a powerful concept that allows TensorFlow to do all sort of things (e.g. automatic differentiation) that are not possible with imperative libraries such as NumPy. But it also comes at the cost of making it harder to grasp. Our attempt here is to demystify TensorFlow and provide some guidelines and best practices for more effective use of TensorFlow.

Let's start with a simple example, we want to multiply two random matrices. First we look at an implementation done in NumPy:


In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:

x = np.random.normal(size=[10, 10])
y = np.random.normal(size=[10, 10])
z = np.dot(x, y)

print(z)

Now we perform the exact same computation this time in TensorFlow:

In [ ]:

x = tf.random_normal([10, 10])
y = tf.random_normal([10, 10])
z = tf.matmul(x, y)

sess = tf.Session()
z_val = sess.run(z)
print(z)
print(z_val)

Unlike NumPy that immediately performs the computation and produces the result, tensorflow only gives us a handle (of type Tensor) to a node in the graph that represents the result. If we try printing the value of z directly, we get something like this:

Tensor("MatMul_1:0", shape=(10, 10), dtype=float32)

Since both the inputs have a fully defined shape, tensorflow is able to infer the shape of the tensor as well as its type. In order to compute the value of the tensor we need to create a session and evaluate it using Session.run() method.

Tip: When using Jupyter notebook make sure to call tf.reset_default_graph() at the beginning to clear the symbolic graph before defining new nodes.

To understand how powerful symbolic computation can be let's have a look at another example. Assume that we have samples from a curve (say f(x) = 5x^2 + 3) and we want to estimate f(x) based on these samples. We define a parametric function g(x, w) = w0 x^2 + w1 x + w2, which is a function of the input x and latent parameters w, our goal is then to find the latent parameters such that g(x, w) ≈ f(x). This can be done by minimizing the following loss function: L(w) =  (f(x) - g(x, w))^2. Although there's a closed form solution for this simple problem, we opt to use a more general approach that can be applied to any arbitrary differentiable function, and that is using stochastic gradient descent. We simply compute the average gradient of L(w) with respect to w over a set of sample points and move in the opposite direction.

Here's how it can be done in TensorFlow:


In [ ]:
tf.reset_default_graph()
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)

w = tf.get_variable("w", shape=[3, 1])
f = tf.stack([tf.square(x), x, tf.ones_like(x)], 1)
yhat = tf.squeeze(tf.matmul(f, w), 1)
loss = tf.nn.l2_loss(yhat - y) + 0.1 * tf.nn.l2_loss(w)

train_op = tf.train.AdamOptimizer(0.1).minimize(loss)

def generate_data():
    x_val = np.random.uniform(-10.0, 10.0, size=100)
    y_val = 5 * np.square(x_val) + 3
    return x_val, y_val

sess = tf.Session()
sess.run(tf.global_variables_initializer())
for _ in range(1000):
    x_val, y_val = generate_data()
    #print(x_val)
    f_cur = tf.stack([tf.square(x_val), x_val, tf.ones_like(x_val)], 1)
    #print(sess.run(f_cur))
    _, loss_val = sess.run([train_op, loss], {x: x_val, y:y_val})
    print(loss_val)
print(sess.run([w]))

## Understanding static and dynamic shapes

Tensors in TensorFlow have a **static shape** attribute which is determined during graph construction. The static shape may be underspecified. For example we might define a tensor of shape [None, 128]:


In [ ]:
a = tf.placeholder(tf.float32, [None, 128])

This means that the first dimension can be of any size and will be determined dynamically during Session.run(). You can **query the static shape of a Tensor as follows**:

In [ ]:
static_shape = a.shape.as_list()      # return [None, 128]
print(static_shape)

To get the **dynamic shape of the tensor** you can call tf.shape op, which returns a tensor representing the shape of the given tensor:

In [ ]:
dynamic_shape = tf.shape(a)
print(dynamic_shape)

The **static shape of a tensor can be set with Tensor.set_shape() method:**

In [ ]:
a.set_shape([32, 128])  # static shape of a is [32, 128]
print(a)
a.set_shape([None, 128])  # first dimension of a is determined dynamically
print(a)

You can reshape a given tensor **dynamically** using tf.reshape function:

In [ ]:
a =  tf.reshape(a, [32, 128])
print(a)

It can be convenient to have a function that **returns the static shape when available and dynamic shape when it's not.** The following utility function does just that:

In [ ]:
def get_shape(tensor):
    static_shape = tensor.shape.as_list()
    print("static:" + str(static_shape))
    dynamic_shape = tf.unstack(tf.shape(tensor))
    print("dynamic:" + str(dynamic_shape))
    dims = [s[1] if s[0] is None else s[0]
            for s in zip(static_shape, dynamic_shape)]
    return dims

Now imagine we want to **convert a Tensor of rank 3 to a tensor of rank 2 by collapsing the second and third dimensions into one**. We can use our get_shape() function to do that:

In [ ]:
b = tf.placeholder(tf.float32, [None, 10, 32])
print("Rank 3 tensor:" + str(b))
shape = get_shape(b)
print(shape)
b = tf.reshape(b, [shape[0], shape[1] * shape[2]])
print(b)

Note that this works whether the shapes are statically specified or not.
In fact we can write a general purpose reshape function to collapse any list of dimensions:

In [ ]:
def reshape(tensor, dims_list):
    shape = get_shape(tensor)
    dims_prod = []
    for dims in dims_list:
        if isinstance(dims, int):
            dims_prod.append(shape[dims])
        elif all([isinstance(shape[d], int) for d in dims]):
            dims_prod.append(np.prod([shape[d] for d in dims]))
        else:
            dims_prod.append(tf.reduce_prod([shape[d] for d in dims]))
    tensor = tf.reshape(tensor, dims_prod)
    return tensor

In [ ]:
b = tf.placeholder(tf.float32, [None, 10, 32])
b = reshape(b, [0, [1, 2]])

## Scopes and when to use them

Variables and tensors in TensorFlow have a name attribute that is used to identify them in the symbolic graph. If you don't specify a name when creating a variable or a tensor, TensorFlow automatically assigns a name for you:

In [ ]:
a = tf.constant(1)
print(a.name)  # prints "Const:0"

b = tf.Variable(1)
print(b.name)  # prints "Variable:0"

You can overwrite the default name by explicitly specifying it:

In [ ]:
a = tf.constant(1, name="a")
print(a.name)  # prints "a:0"

b = tf.Variable(1, name="b")
print(b.name)  # prints "b:0"

TensorFlow introduces two different **context managers** to alter the name of tensors and variables. The first is tf.name_scope:

In [ ]:
with tf.name_scope("scope"):
    a = tf.constant(1, name="a")
    print(a.name)    # prints "scope/a:0"
    
    b = tf.constant(1, name="b")
    print(b.name)    # prints "scope/b:0"
    
    c = tf.get_variable(name="c", shape=[])
    print(c.name)    # prints "c:0"

Note that there are **two ways to define new variables** in TensorFlow, 
- by creating a tf.Variable object or by calling tf.get_variable. 
- Calling tf.get_variable with a new name results in creating a new variable, but if a variable with the same name exists it will raise a ValueError exception, telling us that re-declaring a variable is not allowed.

tf.name_scope affects the name of tensors and variables created with tf.Variable, but doesn't impact the variables created with tf.get_variable.


Unlike tf.name_scope, **tf.variable_scope modifies the name of variables created with tf.get_variable as well:**

In [ ]:
with tf.variable_scope("scope"):
    a = tf.constant(1, name="a")
    print(a.name) # prints "scope/a:0"
    
    b = tf.Variable(1, name="b")
    print(b.name) # prints "scope/b:0"
    
    c = tf.get_variable(name="c", shape=[])
    print(c.name) # prints "scope/c:0"

In [ ]:
with tf.variable_scope("scope"):
    a1 = tf.get_variable(name="a", shape=[])
    a2 = tf.get_variable(name="a", shape=[])  # Disallowed

But what if we actually want to reuse a previously declared variable? Variable scopes also provide the functionality to do that:

In [ ]:
tf.reset_default_graph()
with tf.variable_scope("scope"):
    a1 = tf.get_variable(name="a", shape=[])
with tf.variable_scope("scope", reuse=True):
    a2 = tf.get_variable(name="a", shape=[])  # OK

This becomes handy for example when using built-in neural network layers:

In [ ]:
with tf.variable_scope('my_scope'):
    features1 = tf.layers.conv2d(image1, filters=32, kernel_size=3)
# Use the same convolution weights to process the second image:
with tf.variable_scope('my_scope', reuse=True):
    features2 = tf.layers.conv2d(image2, filters=32, kernel_size=3)

Alternatively you can set reuse to **tf.AUTO_REUSE** which tells TensorFlow to create a new variable if a variable with the same name doesn't exist, and reuse otherwise:

In [ ]:
with tf.variable_scope("scope", reuse=tf.AUTO_REUSE):
    feature1 = tf.layers.conv2d(image1, filters=32, kernel_size=3)
    feature2 = tf.layers.conv2d(image2, filters=32, kernel_size=3)

If you want to do lots of variable sharing keeping track of when to define new variables and when to reuse them can be cumbersome and error prone. tf.AUTO_REUSE simplifies this task but adds the risk of sharing variables that weren't supposed to be shared. 

**TensorFlow templates are another way of tackling the same problem without this risk:**明确说明什么时候进行sharing

In [ ]:
conv3x32 = tf.make_template("conv3x32", lambda x: tf.layers.conv2d(x, 32, 3))
features1 = conv3x32(image1)
features2 = conv3x32(image2)  # Will reuse the convolution weights.

You can turn any function to a TensorFlow template. Upon the first call to a template, the variables defined inside the function would be declared and in the consecutive invocations they would automatically get reused.